In [1]:
from ipycanvas import Canvas, hold_canvas, Path2D

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
import numpy as np

In [9]:
robot_canvas_width = 500
robot_canvas_height = 300
    
robot_canvas = Canvas(width=robot_canvas_width, height=robot_canvas_height)


In [18]:
qpos_x_widget = widgets.FloatSlider(
    value=100,
    min=0,
    max=200.0,
    step=0.1,
    description='Pos (x):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

#pos_x_widget.observe(fx, names='value')

qpos_y_widget = widgets.FloatSlider(
    value=100,
    min=0,
    max=200.0,
    step=0.1,
    description='Pos (x):',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.1f',
)

#pos_y_widget.observe(fy, names='value')


In [22]:
def draw_robot_canvas(canvas, pos_x, pos_y):
    with hold_canvas():
        canvas.clear()
        canvas.reset_transform()

        # The origin of the canvas is the top which is upside down so we will
        # move it to the bottom corner and rotate by 180 degrees.
        # The X-axis will be backwards, but that is better than being upside down.
        canvas.translate(canvas.width,canvas.height)
        canvas.rotate(3.14159)
        
        canvas.line_width = 15
        canvas.line_cap = 'round'
        
        b0 = (20,10)
        j0 = (b0[0], b0[1]+10)
        j1 = (pos_x, pos_y)
        j2 = (300, 200)

        # Draw vertical line for robot base
        canvas.begin_path()
        canvas.stroke_style = "red"
        canvas.stroke_lines([b0,j0])
        canvas.stroke()
        
        # Draw first link (j0-j1)
        canvas.begin_path()
        canvas.stroke_style = "green"
        canvas.stroke_lines([j0,j1])
        canvas.stroke()

        # Draw second link (j1-j2)
        canvas.begin_path()
        canvas.stroke_style = "blue"
        canvas.stroke_lines([j1,j2])
        canvas.stroke()


In [29]:
class robot_configuration:
    j1 = [100,100]

    def update_pos_x(self, pos):
        print (self.j1)
        self.j1[0] = pos
        print (self.j1)
        draw_robot_canvas(robot_canvas, self.j1[0], self.j1[1])
    
    def update_pos_y(self, pos):
        print (self.j1)
        self.j1[1] = pos
        print (self.j1)
        draw_robot_canvas(robot_canvas, self.j1[0], self.j1[1])
        
rc = robot_configuration()

In [30]:
pos_x_widget = interactive(rc.update_pos_x, pos=(0,200))
pos_y_widget = interactive(rc.update_pos_y, pos=(0,200))


In [31]:
display(pos_x_widget)
display(pos_y_widget)
display(robot_canvas)

interactive(children=(IntSlider(value=100, description='pos', max=200), Output()), _dom_classes=('widget-inter…

interactive(children=(IntSlider(value=100, description='pos', max=200), Output()), _dom_classes=('widget-inter…

Canvas(height=300, width=500)